In [96]:
import os
import torch
import pandas as pd
import numpy as np
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.nn.functional import one_hot

import torch
import torch.nn as nn
import torch.optim as optim
#import matplotlib.pyplot as plt

## Load model A

## Load Models B

In [97]:
# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self,dropout_prob=0.5):
        super(SimpleCNN, self).__init__()
        self.dropout_prob = dropout_prob
        
        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, padding=2)
        #self.bn1 = nn.BatchNorm2d(num_features=16)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        
        # Third Convolutional Layer
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.convRes1 = nn.Conv2d(in_channels=16, out_channels = 64, kernel_size=1)

        # Fourth Convolutional Layer
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        # Fifth Convolutional Layer
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)

        # Dropout Layers
        self.dropout1 = nn.Dropout(self.dropout_prob)
        self.dropout2 = nn.Dropout(self.dropout_prob)
        self.dropout3 = nn.Dropout(self.dropout_prob)
        self.dropout4 = nn.Dropout(self.dropout_prob)
        # self.dropout5 = nn.Dropout(self.dropout_prob)

        # Fully Connected Layers
        
        fc1_in_features =  51200  #25600
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=256)
        self.bn2 = nn.BatchNorm1d(num_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.bn3 = nn.BatchNorm1d(num_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=64)
        self.bn4 = nn.BatchNorm1d(num_features=64)
        # self.fc4 = nn.Linear(in_features=64, out_features=32)
        # self.fc5 = nn.Linear(in_features=32, out_features=5)
        self.fc6 = nn.Linear(in_features=64, out_features=5)

        # Output activitation function
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # First Convolutional Block
        # print(x.shape)
        x = self.conv1(x)
        #x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Second Convolutional Block
        # res = x
        # x = self.dropout1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Third Convolutional Block
        # x = self.dropout1(x)
        x = self.conv3(x)
        # print(x.shape)
        # print(res.shape)
        # res = self.convRes1(res)
        # print(res.shape)
        # x += res
        x = self.relu(x)
        x = self.pool(x)

        # Fourth Convolutional Block
        # x = self.dropout1(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.pool(x)

        # Fifth Convolutional Block
        # x = self.dropout1(x)
        x = self.conv5(x)
        x = self.relu(x)
        
        # Flatten for Fully Connected Layers
        x = x.view(x.size(0), -1)
        
        # Fully Connected Layers
        x = self.fc1(x)
        #x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        #x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout1(x)

        x = self.fc3(x)
        #x = self.bn4(x)
        x = self.relu(x)
        x = self.dropout1(x)
        # x = self.fc4(x)
        # x = self.relu(x)
        # x = self.dropout4(x)

        x = self.fc6(x)

        # x = self.fc5(x)but also, there's
        # x = self.softmax(x)
        
        return x

In [98]:
class SimpleCNN_no_dropout(nn.Module):
    def __init__(self):
        super(SimpleCNN_no_dropout, self).__init__()
        self.dropout_prob = 0.5
        
        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        
        # Third Convolutional Layer
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.convRes1 = nn.Conv2d(in_channels=16, out_channels = 64, kernel_size=1)

        # Fourth Convolutional Layer
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        # Fifth Convolutional Layer
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)

        # Dropout Layers
        # self.dropout1 = nn.Dropout(self.dropout_prob)
        # self.dropout2 = nn.Dropout(self.dropout_prob)
        # self.dropout3 = nn.Dropout(self.dropout_prob)
        # self.dropout4 = nn.Dropout(self.dropout_prob)
        # self.dropout5 = nn.Dropout(self.dropout_prob)

        # Fully Connected Layers
        fc1_in_features = 51200
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=256)
        self.bn2 = nn.BatchNorm1d(num_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.bn3 = nn.BatchNorm1d(num_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=64)
        self.bn4 = nn.BatchNorm1d(num_features=64)
        # self.fc4 = nn.Linear(in_features=64, out_features=32)
        # self.fc5 = nn.Linear(in_features=32, out_features=5)
        self.fc6 = nn.Linear(in_features=64, out_features=5)

        # Output activitation function
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # First Convolutional Block
        
        x = self.conv1(x)
        #x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Second Convolutional Block
        #res = x
        # x = self.dropout1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Third Convolutional Block
        # x = self.dropout1(x)
        x = self.conv3(x)
        # print(x.shape)
        # print(res.shape)
        #res = self.convRes1(res)
        # print(res.shape)
        # x += res
        x = self.relu(x)
        x = self.pool(x)

        # Fourth Convolutional Block
        # x = self.dropout1(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.pool(x)

        # Fifth Convolutional Block
        # x = self.dropout1(x)
        x = self.conv5(x)
        x = self.relu(x)
        
        # Flatten for Fully Connected Layers
        x = x.view(x.size(0), -1)
        
        # Fully Connected Layers
        # print(x.shape)
        x = self.fc1(x)
        #x = self.bn2(x)
        x = self.relu(x)
        #x = self.dropout1(x)

        x = self.fc2(x)
        #x = self.bn3(x)
        x = self.relu(x)
        #x = self.dropout1(x)

        x = self.fc3(x)
        #x = self.bn4(x)
        x = self.relu(x)
        #x = self.dropout1(x)
        # x = self.fc4(x)
        # x = self.relu(x)
        # x = self.dropout4(x)

        x = self.fc6(x)

        # x = self.fc5(x)

        # Output layer
        # x = self.softmax(x)
        
        return x

In [158]:
import torch
model_B_reg = torch.load("model B/models/yes_reg_no_augment", weights_only=False, map_location=torch.device('cpu'))
model_B_no_reg = torch.load("model B/models/no_reg_no_augment", weights_only=False, map_location=torch.device('cpu'))
model_B_aug_reg = torch.load("model B/models/yes_reg_yes_augment", weights_only=False, map_location=torch.device('cpu'))
model_B_aug_no_reg = torch.load("model B/models/no_reg_yes_augment", weights_only=False, map_location=torch.device('cpu'))

## Load Val data for model B

In [159]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, annotations_file = None, transform=None, target_transform=None, sex=None, class_num = None):
        self.annotations_file = annotations_file
        self.img_labels = pd.read_csv(annotations_file, names=["file_name","class"]) if (annotations_file != None) else None
        self.img_dir = img_dir
        self.file_names = os.listdir(img_dir)
        if sex: 
            self.file_names = [file_name for file_name in self.file_names if sex in file_name]
            self.img_labels = self.img_labels.loc[self.img_labels['file_name'].str.contains(sex)]
        if class_num:
            self.file_names = [file_name for file_name in self.file_names if str(class_num) == file_name[0]]
            self.img_labels = self.img_labels.loc[self.img_labels['file_name'].str.startswith(str(class_num))]
        display(self.img_labels)
        self.file_names.sort()
        self.transform = transform
        self.target_transform = target_transform
        # display(self.img_labels)
        # display(self.img_labels.loc[self.img_labels['file_name'].str.contains("f")])

    def __len__(self):
        return len(self.img_labels) if self.annotations_file != None else len(self.file_names)

    def __getitem__(self, idx):
        if self.annotations_file != None:
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        else:
            img_path = os.path.join(self.img_dir, self.file_names[idx])
        image = torch.as_tensor(np.load(img_path))
        image = image.unsqueeze(0)
        # print(image[0].shape)
        
        label = one_hot(torch.as_tensor(self.img_labels.iloc[idx, 1]-1),5).unsqueeze(0).to(torch.float) if (self.annotations_file != None) else self.file_names[idx].split(".")[0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [160]:
from torch.utils.data import DataLoader
val_set = CustomImageDataset("model B/preprocessed/val/specs","model B/preprocessed/val/labels.csv")

male_set = CustomImageDataset("model B/preprocessed/val/specs","model B/preprocessed/val/labels.csv", sex='m')
female_set = CustomImageDataset("model B/preprocessed/val/specs","model B/preprocessed/val/labels.csv", sex='f')

class1_set = CustomImageDataset("model B/preprocessed/val/specs","model B/preprocessed/val/labels.csv", class_num = 1)
class2_set = CustomImageDataset("model B/preprocessed/val/specs","model B/preprocessed/val/labels.csv", class_num = 2)
class3_set = CustomImageDataset("model B/preprocessed/val/specs","model B/preprocessed/val/labels.csv", class_num = 3)
class4_set = CustomImageDataset("model B/preprocessed/val/specs","model B/preprocessed/val/labels.csv", class_num = 4)
class5_set = CustomImageDataset("model B/preprocessed/val/specs","model B/preprocessed/val/labels.csv", class_num = 5)

batch_size = 16
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, pin_memory=True)

male_loader = DataLoader(male_set, batch_size=batch_size, shuffle=False, pin_memory=True)
female_loader = DataLoader(female_set, batch_size=batch_size, shuffle=False, pin_memory=True)

class1_loader = DataLoader(class1_set, batch_size=batch_size, shuffle=False, pin_memory=True)
class2_loader = DataLoader(class2_set, batch_size=batch_size, shuffle=False, pin_memory=True)
class3_loader = DataLoader(class3_set, batch_size=batch_size, shuffle=False, pin_memory=True)
class4_loader = DataLoader(class4_set, batch_size=batch_size, shuffle=False, pin_memory=True)
class5_loader = DataLoader(class5_set, batch_size=batch_size, shuffle=False, pin_memory=True)

,file_name,class
0,1f_1026.npy,1
1,1f_1201.npy,1
2,1f_1354.npy,1
3,1f_1355.npy,1
4,1f_1614.npy,1
...,...,...
629,5m_8820.npy,5
630,5m_9417.npy,5
631,5m_9460.npy,5
632,5m_9771.npy,5


,file_name,class
104,1m_1266.npy,1
105,1m_1450.npy,1
106,1m_1843.npy,1
107,1m_2129.npy,1
108,1m_2476.npy,1
...,...,...
629,5m_8820.npy,5
630,5m_9417.npy,5
631,5m_9460.npy,5
632,5m_9771.npy,5


,file_name,class
0,1f_1026.npy,1
1,1f_1201.npy,1
2,1f_1354.npy,1
3,1f_1355.npy,1
4,1f_1614.npy,1
...,...,...
579,5f_9389.npy,5
580,5f_9777.npy,5
581,5f_9912.npy,5
582,5f_9943.npy,5


,file_name,class
0,1f_1026.npy,1
1,1f_1201.npy,1
2,1f_1354.npy,1
3,1f_1355.npy,1
4,1f_1614.npy,1
...,...,...
143,1m_8649.npy,1
144,1m_8954.npy,1
145,1m_9242.npy,1
146,1m_9633.npy,1


,file_name,class
148,2f_1044.npy,2
149,2f_1314.npy,2
150,2f_1748.npy,2
151,2f_2150.npy,2
152,2f_2247.npy,2
...,...,...
268,2m_9802.npy,2
269,2m_9849.npy,2
270,2m_9931.npy,2
271,2m_9984.npy,2


,file_name,class
273,3f_1430.npy,3
274,3f_1594.npy,3
275,3f_1943.npy,3
276,3f_1968.npy,3
277,3f_2015.npy,3
...,...,...
380,3m_9358.npy,3
381,3m_9547.npy,3
382,3m_9760.npy,3
383,3m_9775.npy,3


,file_name,class
385,4f_1006.npy,4
386,4f_1020.npy,4
387,4f_1152.npy,4
388,4f_1170.npy,4
389,4f_1305.npy,4
...,...,...
532,4m_9567.npy,4
533,4m_9609.npy,4
534,4m_9631.npy,4
535,4m_9709.npy,4


,file_name,class
537,5f_1089.npy,5
538,5f_1148.npy,5
539,5f_1582.npy,5
540,5f_1695.npy,5
541,5f_1762.npy,5
...,...,...
629,5m_8820.npy,5
630,5m_9417.npy,5
631,5m_9460.npy,5
632,5m_9771.npy,5


## Basic Evaluation
Accuracy, F1, AUC

In [161]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device set to: {device}")

Device set to: cpu


In [163]:
from torcheval.metrics.functional import multiclass_f1_score
import torch.nn.functional as F
df_val = pd.DataFrame()
from torcheval.metrics.aggregation.auc import AUC

def evaluate_loader(eval_loader, eval_model, name="Loader"):
    with torch.no_grad():
        #set model to eval mode
        eval_model.eval()
        labels = []
        preds = []
        pred_probs = []
        for val_images, val_labels in eval_loader:
            outputs = eval_model(val_images)
            # print(outputs)
            outputs_probs = F.softmax(outputs, dim=1)
            pred_probs += outputs_probs.tolist()
            val_labels = val_labels.reshape(len(val_labels),5)
            val_labels_indices = torch.argmax(val_labels, dim=1)
            labels += val_labels_indices.tolist()
            preds += torch.argmax(outputs,dim=1).tolist()
        
    label_pred = list(zip(labels,preds))
    total = len(label_pred)
    total_correct = sum([1 if a==b else 0 for a,b in label_pred])
    accuracy = total_correct/total
    weighted_macro_f1_score = multiclass_f1_score(torch.stack([torch.tensor(pred) for pred in preds]),torch.stack([torch.tensor(label) for label in labels]), average='weighted', num_classes=5).item()
    print(f"{name}\t-\tacc: {accuracy:.8f}, f1: {weighted_macro_f1_score:.8f}")
    # return accuracy, auc_score, weighted_macro_f1_score
    
for model_name, model in [("Model B Regularized", model_B_reg),("Model B No-Reg", model_B_no_reg),("Model B Augmented No-Reg", model_B_aug_reg),("Model B Augmented No-Reg", model_B_aug_no_reg)]:
    print(f"==== Results for model {model_name} ====")
    evaluate_loader(val_loader, eval_model = model, name = f"All")
    evaluate_loader(male_loader, eval_model = model, name = f"Male")
    evaluate_loader(female_loader, eval_model = model, name = f"Female")
    evaluate_loader(class1_loader, eval_model = model, name = f"Class 1")
    evaluate_loader(class2_loader,eval_model = model,  name = f"Class 2")
    evaluate_loader(class3_loader, eval_model = model, name = f"Class 3")
    evaluate_loader(class4_loader,eval_model = model,  name = f"Class 4")
    evaluate_loader(class5_loader, eval_model = model, name = f"Class 5")
    print()
# Now do the same for class

==== Results for model Model B Regularized ====
All	-	acc: 0.61514196, f1: 0.59783351
Male	-	acc: 0.58169935, f1: 0.56310511
Female	-	acc: 0.64634146, f1: 0.63148963


Class 1	-	acc: 0.92567568, f1: 0.96140355


Class 2	-	acc: 0.56000000, f1: 0.71794873


Class 3	-	acc: 0.64285714, f1: 0.78260875


Class 4	-	acc: 0.59210526, f1: 0.74380165


...

Class 1	-	acc: 0.81756757, f1: 0.89962828


Class 2	-	acc: 0.59200000, f1: 0.74371862


Class 3	-	acc: 0.44642857, f1: 0.61728394


Class 4	-	acc: 0.59868421, f1: 0.74897116


Class 5	-	acc: 0.21649485, f1: 0.35593221

